In [1]:
import sys,os
sys.path.insert(0, os.path.dirname(os.path.realpath('__file__')))
sys.path.insert(1,'../')
from config import conf as cfg
import pandas as pd
from tqdm.notebook import tqdm
import numpy as np
import json
from concurrent.futures import ThreadPoolExecutor, as_completed
import plotly.graph_objects as go
from tools import btools
from IPython.display import HTML
from pandarallel import pandarallel # 导入pandaralle

pandarallel.initialize(progress_bar=False)
%load_ext autoreload
%autoreload 2

INFO: Pandarallel will run on 128 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [4]:
cfg.FILE_WEIGHT_PRODUCTION_BEST_MODEL

'/hpcfs/fhome/shizhenkun/codebase/RXNRECer/data/model/production_185846best.pth'